# Progress Report 2

Natasha Kamtekar 
nak142

In [1]:
# Libraries
import nltk
import numpy as np
import pandas as pd
import json
import re
from pandas.io.json import json_normalize
import pickle

In [2]:
# Formatting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%pprint

Pretty printing has been turned OFF


First i'm going to unpickle all the necessary data I need for this notebook. I went ahead and pickled the data I worked on in [progress report 1](https://github.com/Data-Science-for-Linguists-2020/Twitter-Positivity-Analysis/blob/master/notebooks/data_parsing.ipynb) as well as the datasets for the [classifier]() I generated. 

I'm flashing the 2011/2019 tweets, but setting the classifier data aside until later. 

In [3]:
f = open('tweets2011.pkl', 'rb')      
tweets2011 = pickle.load(f) 
f.close() 

In [4]:
f = open('tweets2019.pkl', 'rb')      
tweets2019 = pickle.load(f) 
f.close() 

In [5]:
f = open('classifier.pkl', 'rb')      
classifier = pickle.load(f) 
f.close() 

In [6]:
f = open('word_features.pkl', 'rb')      
word_features = pickle.load(f) 
f.close() 

In [7]:
tweets2019
tweets2011

,text,user.lang,created_at
0,RT @yukichii927: 起きた？おはよう✨\n\n昨日思いがけず、沢山のコメントを...,ja,Wed May 01 06:29:00 +0000 2019
1,(ولكل أمة أجل فإذا جاء أجلهم لا يستأخرون ساعة ...,ar,Wed May 01 06:29:00 +0000 2019
2,RT @elconfidencial: La chapuza de la operación...,en,Wed May 01 06:29:00 +0000 2019
3,RT @bpswag27: ไถทวิตแต่ไม่กดเข้าเดม มีอยู่จริง...,th,Wed May 01 06:29:00 +0000 2019
4,@onion__017 汚ねぇ元気玉😂www,en,Wed May 01 06:29:00 +0000 2019
...,...,...,...
6462,حسبي الله لا إله إلا هو عليه توكلت وهو رب العر...,en,Wed May 01 06:31:45 +0000 2019
6463,@llixa_16 اواههه يكثرهم عند ربي,ar,Wed May 01 06:31:45 +0000 2019
6464,แอคทีฟอยู่ไม่กี่แอพ ทวิท ไอจี ยูทูป จบ,th,Wed May 01 06:31:45 +0000 2019
6465,Ewan,en,Wed May 01 06:31:45 +0000 2019


,text,user.lang,created_at
0,"@firawidya iya fir,hihi. Rencananya awal bulan...",en,Wed Sep 28 01:48:17 +0000 2011
1,"voo sair aki sem maldade, to com mt sono ; #fato",pt,Wed Sep 28 01:48:17 +0000 2011
2,"RT @TheNoteboook: If he loves you, he'll care ...",en,Wed Sep 28 01:48:17 +0000 2011
3,Ombbb @ddlavato singing on abc!,en,Wed Sep 28 01:48:17 +0000 2011
4,RT @aniserra: Nunca voy a entender como sabe e...,es,Wed Sep 28 01:48:15 +0000 2011
...,...,...,...
3691,@Agosbonelli a full!!! Después empiezan las ve...,es,Wed Sep 28 02:38:59 +0000 2011
3692,"@hay_ross ahaha, so am I, but we have our reas...",en,Wed Sep 28 02:38:59 +0000 2011
3693,@densukefukusima つまり、プロローグを消し、回想シーンを冒頭にすることで一工...,ja,Wed Sep 28 02:38:59 +0000 2011
3694,«@Socialite_Trina ♪ Rolex/ More sex/good weed/...,en,Wed Sep 28 02:38:59 +0000 2011


## Cleaning Up The Data

I have worked with a limited amount of my total data in order to make data processing go faster for my poor laptop. However, after testing the limits of my processor while I built the classifier, I want to add more to this notebook when I get the chance.

As you can see, there are a lot of different languages in this dataset. The user.lang field in the dataframe does not describe the language of the tweet, but describes the users settings. This can cause us to keep data that isnt english and toss out actual english data. 

In order to clean that up i'm using the [langdetect](https://pypi.org/project/langdetect/) library. 

In [8]:
from langdetect import detect

I am creating a dataframe to store the predicted languages in, cleaning up the text using regex, and then appending the language dataframe to the original tweet dataframe. I do this for both the 2011 and 2019 sets. 

In [9]:
i = 0
lng = pd.DataFrame()
while True: 
    try:
        tweets2011.loc[i, "text"] = re.sub(r'(?:(@|&|;|http|https)[\w_]+)', '', tweets2011.loc[i, "text"]) #cleaning
        tmp = pd.DataFrame({'languages': [detect(tweets2011.loc[i, "text"])]}) 
        lng = lng.append(tmp, sort=False, ignore_index=True)
        if i > len(tweets2011):
            break
        i = i+1
    except:
        tmp = pd.DataFrame({'languages': ["error"]}) #If the language is not able to be predicted
        lng = lng.append(tmp, sort=False, ignore_index=True)
        if i > len(tweets2011):
            break
        i = i+1


In [10]:
# This is the same as above, but for 2019
i = 0
lng = pd.DataFrame()
while True: 
    try:
        tweets2019.loc[i, "text"] = re.sub(r'(?:@[\w_]+)', '', tweets2019.loc[i, "text"])
        tmp = pd.DataFrame({'languages': [detect(tweets2019.loc[i, "text"])]})
        lng = lng.append(tmp, sort=False, ignore_index=True)
        if i > len(tweets2019):
            break
        i = i+1
    except:
        tmp = pd.DataFrame({'languages': ["error"]})
        lng = lng.append(tmp, sort=False, ignore_index=True)
        if i > len(tweets2011):
            break
        i = i+1


In [11]:
tweets2011['languages'] = lng
tweets2011 = tweets2011.drop(columns=['user.lang'])
tweets2019['languages'] = lng
tweets2019 = tweets2019.drop(columns=['user.lang'])

Here is a breakdown of the top 5 languages represented during each year.

In [12]:
tweets2011['languages'].value_counts(5)
tweets2019['languages'].value_counts(5)

en       0.250812
ja       0.213474
error    0.149621
th       0.066288
ko       0.048431
ar       0.047619
es       0.042208
id       0.021104
pt       0.018939
tl       0.014340
tr       0.013528
fr       0.010552
ca       0.009470
hi       0.008658
de       0.008117
it       0.006764
so       0.006223
vi       0.006223
nl       0.006223
ru       0.005682
fa       0.003517
sw       0.003517
et       0.003517
no       0.003247
ur       0.003247
af       0.002706
cy       0.002706
pl       0.002435
ta       0.002165
el       0.002165
ro       0.002165
fi       0.001623
hu       0.001623
sv       0.001623
hr       0.001082
da       0.001082
zh-cn    0.000812
sq       0.000812
lt       0.000541
ne       0.000541
lv       0.000541
bg       0.000541
ml       0.000541
zh-tw    0.000541
mk       0.000541
uk       0.000541
sl       0.000271
he       0.000271
bn       0.000271
te       0.000271
sk       0.000271
Name: languages, dtype: float64

en       0.252153
ja       0.213402
error    0.149085
th       0.065931
ko       0.048439
ar       0.047363
es       0.042519
id       0.021259
pt       0.018837
tl       0.014263
tr       0.013455
fr       0.010495
ca       0.009419
hi       0.008611
de       0.008073
it       0.006728
so       0.006189
vi       0.006189
nl       0.006189
ru       0.005651
et       0.003498
sw       0.003498
fa       0.003498
ur       0.003229
no       0.003229
af       0.002691
cy       0.002691
pl       0.002422
ro       0.002422
ta       0.002153
el       0.002153
hu       0.001615
fi       0.001615
sv       0.001615
da       0.001076
hr       0.001076
sq       0.000807
zh-cn    0.000807
lt       0.000538
mk       0.000538
uk       0.000538
lv       0.000538
ml       0.000538
zh-tw    0.000538
bg       0.000538
ne       0.000538
sk       0.000269
te       0.000269
sl       0.000269
he       0.000269
bn       0.000269
Name: languages, dtype: float64

Now I am paring down the dataframe to only have english as the language. It is not 100% accurate, but it judges the tweet rather than the user. 

In [13]:
tweets2011 = tweets2011[tweets2011['languages'] == 'en']
len(tweets2011[tweets2011['languages'] == 'en'])
tweets2011.reset_index()

927

,index,text,created_at,languages
0,8,Made a Wendy's pit stop,Wed Sep 28 01:48:18 +0000 2011,en
1,12,No sappy love songs tn... just please.,Wed Sep 28 01:48:18 +0000 2011,en
2,18,knp siyy hrs nge ping berkali2 (⌣́_⌣̀),Wed Sep 28 01:48:18 +0000 2011,en
3,19,"""eu te amo"" verdadeiros sempre são bem-vindos",Wed Sep 28 01:48:18 +0000 2011,en
4,21,#JevaVacana♥,Wed Sep 28 01:48:18 +0000 2011,en
...,...,...,...,...
922,3665,haha thats weird af. Cuz i said haly would be...,Wed Sep 28 02:38:58 +0000 2011,en
923,3667,saludos afectuosos!!!!,Wed Sep 28 02:38:58 +0000 2011,en
924,3668,"RT : My favorite kinds of movies: Disney, Pixa...",Wed Sep 28 02:38:58 +0000 2011,en
925,3670,"Beeeeeeija aeew , queremos show\n ( live on ht...",Wed Sep 28 02:38:58 +0000 2011,en


In [14]:
tweets2019 = tweets2019[tweets2019['languages'] == 'en']
len(tweets2019[tweets2019['languages'] == 'en'])
tweets2019.reset_index()

937

,index,text,created_at,languages
0,8,"RT : Rep. Ted Lieu, member of the House Judici...",Wed May 01 06:29:00 +0000 2019,en
1,12,RT : https://t.co/oTwwTOmEzD,Wed May 01 06:29:00 +0000 2019,en
2,18,RT : Happy birthday to the man of steel.. Our ...,Wed May 01 06:29:00 +0000 2019,en
3,19,I give up on these guys,Wed May 01 06:29:00 +0000 2019,en
4,21,lmaooo the accuracy,Wed May 01 06:29:00 +0000 2019,en
...,...,...,...,...
932,3703,"For Black Ice Booking, Features and Booking St...",Wed May 01 06:30:33 +0000 2019,en
933,3705,RT : https://t.co/gunEkOlQoQ,Wed May 01 06:30:33 +0000 2019,en
934,3708,Wish me luck ㅠㅠㅠ,Wed May 01 06:30:33 +0000 2019,en
935,3710,Come to Dubai then 😋,Wed May 01 06:30:33 +0000 2019,en


## Analyzing The Data

Now we are going to analyze the tweets in the first of two ways: Frequency Distrubution

In order to do that i'm going to tokenize all the tweets for each year.

In [15]:
from nltk import word_tokenize

In [16]:
toks2011 = [word_tokenize(i) for i in tweets2011['text']]
toks2019 = [word_tokenize(i) for i in tweets2019['text']]

In [17]:
toks2011[:5]
toks2019[:5]

[['Made', 'a', 'Wendy', "'s", 'pit', 'stop'], ['No', 'sappy', 'love', 'songs', 'tn', '...', 'just', 'please', '.'], ['knp', 'siyy', 'hrs', 'nge', 'ping', 'berkali2', '(', '⌣́_⌣̀', ')'], ['``', 'eu', 'te', 'amo', "''", 'verdadeiros', 'sempre', 'são', 'bem-vindos'], ['#', 'JevaVacana♥']]

[['RT', ':', 'Rep.', 'Ted', 'Lieu', ',', 'member', 'of', 'the', 'House', 'Judiciary', 'Committee', ',', 'tells', 'CNN', "'s", 'that', 'if', 'Attorney', 'General', 'Bill', 'Barr', 'does', 'not', 'a…'], ['RT', ':', 'https', ':', '//t.co/oTwwTOmEzD'], ['RT', ':', 'Happy', 'birthday', 'to', 'the', 'man', 'of', 'steel..', 'Our', '#', 'ThalaAjith', 'Sir', '...', 'May', 'u', 'haveee', 'a', 'lonnnngggggggg', 'lonnngggggggg', 'life', 'filled', 'with', 'loa…'], ['I', 'give', 'up', 'on', 'these', 'guys'], ['lmaooo', 'the', 'accuracy']]

`toks2011` and `toks2019` are in a nested list format, and in order to check the frequency distrubution of all text in either year we need to flatten them into one list. The following code does this.

In [18]:
flat_2011_toks = []
flat_2019_toks = []
def remove2011Nesting(l): 
    for i in l: 
        if type(i) == list: 
            remove2011Nesting(i) 
        else: 
            flat_2011_toks.append(i) 
            
def remove2019Nesting(l): 
    for i in l: 
        if type(i) == list: 
            remove2019Nesting(i) 
        else: 
            flat_2019_toks.append(i)
  
remove2011Nesting(toks2011) 
remove2019Nesting(toks2019)

Now I am removing all punctuation, blank elements, and twitter specific lingo like "RT" for retweet

In [19]:
import string
table = str.maketrans('', '', string.punctuation)

flat_2011_toks = [w.translate(table) for w in flat_2011_toks]
flat_2019_toks = [w.translate(table) for w in flat_2019_toks]

In [20]:
noRT = str.maketrans('', '', 'RT')
flat_2011_toks = [w.translate(noRT) for w in flat_2011_toks]
flat_2019_toks = [w.translate(noRT) for w in flat_2019_toks]

In [21]:
while("" in flat_2011_toks): 
    flat_2011_toks.remove("") 
    
while("" in flat_2019_toks): 
    flat_2019_toks.remove("")
    
print(flat_2011_toks[:10])   
print(flat_2019_toks[:10])

['Made', 'a', 'Wendy', 's', 'pit', 'stop', 'No', 'sappy', 'love', 'songs']
['ep', 'ed', 'Lieu', 'member', 'of', 'the', 'House', 'Judiciary', 'Committee', 'tells']


Perfect! Now we can take the frequency distrubution of both datasets

In [22]:
fd2011 = nltk.FreqDist(flat_2011_toks)
fd2011

fd2019 = nltk.FreqDist(flat_2019_toks)
fd2019

FreqDist({'I': 141, 'a': 136, 'to': 97, 'http': 96, 'the': 87, 'que': 72, 'de': 70, 'me': 69, 'you': 62, 'of': 55, ...})

FreqDist({'https': 463, 'the': 299, 'to': 266, 'a': 206, 'and': 184, '’': 171, 'I': 163, 'you': 158, 'of': 153, 'is': 127, ...})

Here are the most common words...

In [23]:
fd2019.most_common(70)
fd2011.most_common(70)

[('https', 463), ('the', 299), ('to', 266), ('a', 206), ('and', 184), ('’', 171), ('I', 163), ('you', 158), ('of', 153), ('is', 127), ('s', 124), ('for', 109), ('in', 98), ('on', 91), ('this', 88), ('he', 86), ('it', 83), ('my', 81), ('⠀', 80), ('that', 76), ('me', 67), ('with', 61), ('BBMAsopSocial', 58), ('BS', 56), ('his', 55), ('was', 54), ('at', 52), ('be', 50), ('…', 49), ('i', 48), ('1', 44), ('are', 41), ('so', 41), ('not', 40), ('t', 39), ('if', 38), ('amp', 38), ('we', 38), ('do', 38), ('your', 37), ('from', 36), ('nt', 35), ('all', 35), ('can', 35), ('see', 34), ('m', 34), ('like', 33), ('have', 33), ('how', 32), ('up', 31), ('but', 31), ('when', 29), ('who', 29), ('out', 28), ('“', 28), ('re', 28), ('will', 28), ('one', 28), ('just', 27), ('her', 25), ('”', 24), ('they', 24), ('people', 24), ('as', 23), ('than', 22), ('My', 22), ('by', 22), ('what', 21), ('want', 21), ('here', 20)]

[('I', 141), ('a', 136), ('to', 97), ('http', 96), ('the', 87), ('que', 72), ('de', 70), ('me', 69), ('you', 62), ('of', 55), ('and', 54), ('it', 53), ('my', 50), ('no', 48), ('y', 41), ('is', 40), ('that', 38), ('s', 37), ('on', 37), ('o', 34), ('in', 34), ('nt', 34), ('like', 33), ('el', 32), ('do', 32), ('m', 29), ('te', 28), ('i', 28), ('la', 27), ('e', 27), ('be', 26), ('for', 26), ('he', 26), ('just', 25), ('so', 25), ('not', 24), ('this', 24), ('have', 24), ('with', 22), ('por', 22), ('lol', 22), ('at', 21), ('é', 21), ('se', 20), ('your', 20), ('en', 20), ('what', 20), ('u', 19), ('es', 19), ('eu', 18), ('when', 18), ('get', 18), ('tuts', 18), ('but', 17), ('got', 17), ('if', 17), ('A', 17), ('da', 17), ('out', 16), ('can', 16), ('are', 16), ('q', 16), ('un', 16), ('as', 15), ('they', 15), ('all', 15), ('time', 15), ('shit', 14), ('na', 14), ('ll', 14)]

Seems like there are still some foreign words that slipped through langdetect, something to work on for the future. 
Just taking a look at the most common for the two years I can see that most of the non-english words seem to be concentrated in 2011 and kpop is more talked about in 2019 ("BBMAs", "BTS" being indicators). These are interesting to note, but don't get at the question I want to be asking with my sentiment analysis.

So I compared frequencies of certain words occurring in 2011 versus 2019. 

In [24]:
print("Frequency of...\n")
print("The word \"u\" in 2011:",fd2011["u"], "versus 2019:",fd2019["u"])
print("The word \"lol\" in 2011:",fd2011["lol"], "versus 2019:",fd2019["lol"])
print('\n')
print("The word \"excited\" in 2011:",fd2011["excited"], "versus 2019:",fd2019["excited"])
print("The word \"happy\" in 2011:",fd2011["happy"], "versus 2019:",fd2019["happy"])
print("The word \"depressed\" in 2011:",fd2011["depressed"], "versus 2019:",fd2019["depressed"])
print("The word \"hope\" in 2011:",fd2011["hope"], "versus 2019:",fd2019["hope"])
print("The word \"love\" in 2011:",fd2011["love"], "versus 2019:",fd2019["love"])
print("The word \"hate\" in 2011:",fd2011["hate"], "versus 2019:",fd2019["hate"])
print('\n')
print("The word \"they\" in 2011:",fd2011["they"], "versus 2019:",fd2019["they"])
print("The word \"transgender\" in 2011:",fd2011["transgender"], "versus 2019:",fd2019["transgender"])
print("The word \"election\" in 2011:",fd2011["election"], "versus 2019:",fd2019["election"])
print("The word \"race\" in 2011:",fd2011["race"], "versus 2019:",fd2019["race"])
print("The word \"gay\" in 2011:",fd2011["gay"], "versus 2019:",fd2019["gay"])

Frequency of...

The word "u" in 2011: 19 versus 2019: 14
The word "lol" in 2011: 22 versus 2019: 3


The word "excited" in 2011: 2 versus 2019: 1
The word "happy" in 2011: 3 versus 2019: 2
The word "depressed" in 2011: 0 versus 2019: 1
The word "hope" in 2011: 2 versus 2019: 3
The word "love" in 2011: 11 versus 2019: 20
The word "hate" in 2011: 5 versus 2019: 3


The word "they" in 2011: 15 versus 2019: 24
The word "transgender" in 2011: 0 versus 2019: 1
The word "election" in 2011: 0 versus 2019: 1
The word "race" in 2011: 2 versus 2019: 0
The word "gay" in 2011: 1 versus 2019: 2


I divided them up into a few groups:
- Texting lingo
- "Feeling" words
- Current topics of conversation in pop culture

For texting lingo, I was surprised at how completely "lol" has gone out of style compared to 2011. From my own observations, "lol" is used more sarcasticly and has a negative connotation these days, but I was definitely expecting to see it. "u" as a substitue for "you" still seems to be standing strong against the change of time. 

With "feeling" words I targeted words that are used to describe sentiment towards an object or person. The data was too sparse to make any notable connections, other than for the word "love" where 2019 triumphs. My guess for why this could be is that there is a cultural "self care" renaissance happening online where people are taking the time to think about mental health. Maybe this increase in love comes from that trend?

Finally, the current events section is where I targetted words that are "buzz words" in this day and age and are talked about on political twitter. The one that surprised me was the increase in the pronoun "they". In the past 10 years there has been a spotlight on transgender issues, namely using "they" as a gender neutral pronoun. My guess is that conversation promoted the increase in use of that pronoun.

However, this look into the frequecy of words leaves a lot to be desired. I also wanted to create bigrams to take a closer look at the context of some of these words.

In [25]:
bigram2011 = list(nltk.ngrams(flat_2011_toks, 2))
bigram2019 = list(nltk.ngrams(flat_2019_toks, 2))

In [26]:
bigram2011fd = nltk.FreqDist(bigram2011)
bigram2019fd = nltk.FreqDist(bigram2019)

In [27]:
bigram2011fd.most_common(50)
bigram2019fd.most_common(50)

[(('I', 'm'), 27), (('tuts', 'tuts'), 17), (('do', 'nt'), 12), (('of', 'the'), 9), (('to', 'be'), 8), (('a', 'la'), 7), (('I', 'll'), 7), (('I', 'do'), 7), (('on', 'the'), 7), (('of', 'my'), 6), (('I', 'can'), 6), (('in', 'the'), 6), (('it', 's'), 6), (('wan', 'na'), 5), (('this', 'is'), 5), (('for', 'the'), 5), (('going', 'to'), 5), (('need', 'to'), 5), (('I', 'want'), 5), (('want', 'to'), 5), (('I', 'got'), 4), (('if', 'you'), 4), (('that', 's'), 4), (('you', 're'), 4), (('in', 'my'), 4), (('got', 'ta'), 4), (('Do', 'nt'), 4), (('to', 'do'), 4), (('have', 'a'), 4), (('when', 'I'), 4), (('I', 'am'), 4), (('si', 'no'), 4), (('que', 'no'), 4), (('o', 'que'), 4), (('is', 'a'), 4), (('que', 'é'), 4), (('en', 'la'), 4), (('te', 'amo'), 3), (('out', 'and'), 3), (('it', 'is'), 3), (('but', 'I'), 3), (('how', 'to'), 3), (('needs', 'to'), 3), (('de', 'tudo'), 3), (('que', 'tem'), 3), (('at', 'the'), 3), (('not', 'be'), 3), (('my', 'life'), 3), (('to', 'a'), 3), (('to', 'me'), 3)]

[(('⠀', '⠀'), 72), (('’', 's'), 63), (('’', 't'), 38), (('BBMAsopSocial', 'BS'), 31), (('of', 'the'), 26), (('I', '’'), 25), (('in', 'the'), 22), (('BS', 'https'), 18), (('’', 'm'), 18), (('BBMAsopSocial', 'EXO'), 17), (('is', 'a'), 17), (('if', 'you'), 16), (('’', 're'), 15), (('I', 'm'), 15), (('his', 'is'), 14), (('to', 'be'), 14), (('do', 'nt'), 14), (('If', 'you'), 13), (('to', 'see'), 12), (('for', 'the'), 12), (('to', 'the'), 11), (('on', 'the'), 11), (('you', '’'), 11), (('with', 'the'), 11), (('1', 'Vote'), 10), (('is', 'the'), 9), (('from', 'the'), 9), (('in', 'a'), 9), (('don', '’'), 9), (('it', 'to'), 8), (('all', 'the'), 8), (('the', 'best'), 8), (('want', 'to'), 8), (('it', '’'), 8), (('…', 'https'), 8), (('and', 'it'), 8), (('EXO', 'https'), 8), (('but', 'I'), 8), (('this', 'is'), 8), (('Happy', 'birthday'), 7), (('you', 'are'), 7), (('going', 'to'), 7), (('to', 'get'), 7), (('to', 'vote'), 7), (('on', 'my'), 7), (('have', 'a'), 7), (('it', 'was'), 7), (('BBMAsAchievemen

In [28]:
print("Frequency of...\n")
print("The word \"don't\" in 2011:",bigram2011fd[('do', 'nt')], "versus 2019:",bigram2019fd[('do', 'nt')])
print("The word \"can't\" in 2011:",bigram2011fd[('ca', 'nt')], "versus 2019:",bigram2019fd[('ca', 'nt')])
print("The word \"won't\" in 2011:",bigram2011fd[('wo', 'nt')], "versus 2019:",bigram2019fd[('wo', 'nt')])
print("The word \"aren't\" in 2011:",bigram2011fd[('are', 'nt')], "versus 2019:",bigram2019fd[('are', 'nt')])
print("The word \"ain't\" in 2011:",bigram2011fd[('ai', 'nt')], "versus 2019:",bigram2019fd[('ai', 'nt')])

Frequency of...

The word "don't" in 2011: 12 versus 2019: 14
The word "can't" in 2011: 2 versus 2019: 4
The word "won't" in 2011: 3 versus 2019: 0
The word "aren't" in 2011: 1 versus 2019: 1
The word "ain't" in 2011: 2 versus 2019: 1


I looked at the frequency of negations between the two years. For the most part, there are no glaring differences between the two eras. Let's look at more substansial tuples

In [29]:
print("The words \"love you\" in 2011:",bigram2011fd[('love', 'you')], "versus 2019:",bigram2019fd[('love', 'you')])
print("The words \"love you\" in 2011:",bigram2011fd[('love', 'myself')], "versus 2019:",bigram2019fd[('love', 'myself')])
print("The word \"hate it\" in 2011:",bigram2011fd[('hate', 'it')], "versus 2019:",bigram2019fd[('hate', 'it')])
print('\n')
print("The word \"I can't\" in 2011:",bigram2011fd[('I', 'ca')], "versus 2019:",bigram2019fd[('I', 'ca')])
print("The word \"voting for\" in 2011:",bigram2011fd[('voting', 'for')], "versus 2019:",bigram2019fd[('voting', 'for')])
print('\n')
print("The word \"can do\" in 2011:",bigram2011fd[('can', 'do')], "versus 2019:",bigram2019fd[('can', 'do')])
print("The word \"can do\" in 2011:",bigram2011fd[('nothing', 'i')], "versus 2019:",bigram2019fd[('nothing', 'i')])
print("The word \"ca/wo/do n't do\" in 2011:",bigram2011fd[('nt', 'do')], "versus 2019:",bigram2019fd[('nt', 'do')])

The words "love you" in 2011: 1 versus 2019: 4
The words "love you" in 2011: 0 versus 2019: 0
The word "hate it" in 2011: 1 versus 2019: 0


The word "I can't" in 2011: 0 versus 2019: 1
The word "voting for" in 2011: 0 versus 2019: 4


The word "can do" in 2011: 1 versus 2019: 4
The word "can do" in 2011: 0 versus 2019: 0
The word "ca/wo/do n't do" in 2011: 0 versus 2019: 0


In this section I looked at tuples, building off of some of the single terms I looked at earlier. The use of "love you" slightly increased since 2011, however there are no instances of "love myself" which disproves my hypothesis about the "self care" trend.

Interestingly the usage of "can do" also increased, implying that tweets have become more hopeful ("nothing i" shows that the phrase isn't "nothing i can do" or something similar). 

Additionally, the bigram "voting for" going from 0 to 4 reflects the time period for these tweets well. 

***


Now finally the data we unpickled for the classifier is going to come into play here. I am redefining the method to create document_features so that it can be used in this notebook.

In [30]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [42]:
twt1 = document_features(toks2011[192])
twt2 = document_features(toks2011[76]) 
twt3 = document_features(toks2011[23]) 
twt4 = document_features(toks2011[892])
twt5 = document_features(toks2011[677])
twt6 = document_features(toks2011[655]) 
twt7 = document_features(toks2011[793]) 
twt8 = document_features(toks2011[93])
print("Classification of",toks2011[192],"\nis",classifier.classify(twt1))
print("Classification of", toks2011[76],"\nis",classifier.classify(twt2))
print("Classification of", toks2011[23],"\nis",classifier.classify(twt3))
print("Classification of", toks2011[892],"\nis",classifier.classify(twt4))
print("Classification of",toks2011[677],"\nis",classifier.classify(twt5))
print("Classification of", toks2011[655],"\nis",classifier.classify(twt6))
print("Classification of", toks2011[793],"\nis",classifier.classify(twt7))
print("Classification of", toks2011[93],"\nis",classifier.classify(twt8))

Classification of ['RT', ':', 'Live', 'everyday', 'as', 'if', 'we', "'re", 'waking', 'up', 'for', 'the', 'first', 'time', '.', 'Appreciate', 'life', 'as', 'if', 'it', "'s", 'the', 'last', 'time', 'we', "'re", 'going', 'to', 'sleep', '.'] 
is 4
Classification of ['RT', ':', 'Thanks', 'for', 'coming', '!', '!', 'Glad', 'to', 'hear', 'you', 'enjoyed', 'it', '.'] 
is 4
Classification of ['I', 'never', 'talked', 'to', 'a', 'female', 'outside', 'my', 'race', 'so', 'this', 'is', 'very', 'new', 'to', 'me', 'o_O'] 
is 0
Classification of ['...', 'no', '.', 'But', 'add', 'ambition', 'to', 'that', 'and', 'experience', '=', 'unstoppable', ',', 'happy', ',', 'success', '!'] 
is 4
Classification of ['Gahh.i', 'want', 'both', '...', 'yoontae..but', 'now', 'i', 'want', 'Yul', 'pic', '!', '!', '!'] 
is 0
Classification of ['I', 'swear', 'I', 'treat', 'my', 'car', 'like', 'a', 'sports', 'car', '!'] 
is 4
Classification of ['i', 'got', 'ta', 'be', 'a', 'mid-20', "'s", 'woman..errbody', 'wonderin', 'y', '

Here is a quick look at the classifier, it seems to be getting the tweets somewhat correct. My next step will be to improve the classifier's accuracy before I perform an analysis on the tweets from 2011 and 2019 to see which of them are more positive or negative respectively. 